In [30]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [31]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

In [32]:
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [33]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers) #request package to retrieve the webpage
        self.body = response.content #get the content of the webpage
        soup = BeautifulSoup(self.body, 'html.parser') #parse the content with BeautifulSoup
        self.title = soup.title.string if soup.title else "No title found" 
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]): #remove script, style, img, input tags
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')] #extract all hyperlinks
        self.links = [link for link in links if link] 

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [34]:
site=Website("https://www.apple.com/")
print(site.get_contents())

Webpage Title:
Apple
Webpage Contents:
Apple
Apple
Store
Mac
iPad
iPhone
Watch
Vision
AirPods
TV & Home
Entertainment
Accessories
Support
0
+
Last chance to get AirPods or an eligible accessory of your choice when you buy Mac or iPad with education savings. Ends 9.30.
1
Shop
iPhone 17 Pro
All out Pro.
Learn more
Buy
iPhone Air
The thinnest iPhone ever. With the power of pro inside.
Learn more
Buy
iPhone 17
Magichromatic.
Learn more
Buy
AirPods Pro 3
The world’s best in-ear Active Noise Cancellation.
Learn more
Buy
Apple Watch Series 11
The ultimate way to watch your health.
Learn more
Buy
Apple Watch SE 3
Walk it. Talk it. Track it. Love it.
Learn more
Buy
Apple Watch Ultra 3
Personal beast.
Learn more
Buy
Any condition carrier deals are here.
Select carriers accept eligible trade-in devices in any condition. Other offers available.
2
Find your deal
Apple Trade In
Get up to $200–$700 in credit when you trade in iPhone 13 or higher.
3
Get your estimate
Apple TV+
FAM Gallery
Watch now
St

In [35]:
print(site.links)

['/', '/us/shop/goto/store', '/mac/', '/ipad/', '/iphone/', '/watch/', '/apple-vision-pro/', '/airpods/', '/tv-home/', '/entertainment/', '/us/shop/goto/buy_accessories', 'https://support.apple.com/?cid=gn-ols-home-hp-tab', '/us/search', '/us/shop/goto/bag', '#footnote-1', '/us-edu/shop/goto/edu_store', '/iphone-17-pro/', '/iphone-17-pro/', '/us/shop/goto/buy_iphone/iphone_17_pro', '/iphone-air/', '/iphone-air/', '/us/shop/goto/buy_iphone/iphone_air', '/iphone-17/', '/iphone-17/', '/us/shop/goto/buy_iphone/iphone_17', '/airpods-pro/', '/airpods-pro/', '/us/shop/goto/buy_airpods/airpods_pro_3', '/apple-watch-series-11/', '/apple-watch-series-11/', '/us/shop/goto/buy_watch/apple_watch_series_11', '/apple-watch-se-3/', '/apple-watch-se-3/', '/us/shop/goto/buy_watch/apple_watch_se', '/apple-watch-ultra-3/', '/apple-watch-ultra-3/', '/us/shop/goto/buy_watch/apple_watch_ultra_3', '/us/shop/goto/buy_iphone/carrier_offers', '/us/shop/goto/buy_iphone/carrier_offers', '/us/shop/goto/trade_in', '

In [36]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [37]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}



In [38]:
def get_links_user_prompt(website): #website is an instance of the Website class
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [39]:
print(get_links_user_prompt(site))

Here is the list of links on the website of https://www.apple.com/ - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
/
/us/shop/goto/store
/mac/
/ipad/
/iphone/
/watch/
/apple-vision-pro/
/airpods/
/tv-home/
/entertainment/
/us/shop/goto/buy_accessories
https://support.apple.com/?cid=gn-ols-home-hp-tab
/us/search
/us/shop/goto/bag
#footnote-1
/us-edu/shop/goto/edu_store
/iphone-17-pro/
/iphone-17-pro/
/us/shop/goto/buy_iphone/iphone_17_pro
/iphone-air/
/iphone-air/
/us/shop/goto/buy_iphone/iphone_air
/iphone-17/
/iphone-17/
/us/shop/goto/buy_iphone/iphone_17
/airpods-pro/
/airpods-pro/
/us/shop/goto/buy_airpods/airpods_pro_3
/apple-watch-series-11/
/apple-watch-series-11/
/us/shop/goto/buy_watch/apple_watch_series_11
/apple-watch-se-3/
/apple-watch-se-3/
/us/shop/goto/buy_watch/apple_watch_se
/apple-watch-ultr

In [40]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content 
    return json.loads(result)

In [41]:
get_links("https://www.apple.com/")

{'links': [{'type': 'about page', 'url': 'https://www.apple.com/about/'},
  {'type': 'careers page', 'url': 'https://www.apple.com/careers/us/'}]}

In [42]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents() #get contents of each relevant link
    return result

In [43]:
print(get_all_details("https://www.apple.com/"))

Found links: {'links': [{'type': 'about page', 'url': 'https://www.apple.com/about/'}, {'type': 'careers page', 'url': 'https://www.apple.com/careers/us/'}, {'type': 'company page', 'url': 'https://www.apple.com/newsroom/'}, {'type': 'leadership page', 'url': 'https://www.apple.com/leadership/'}]}
Landing page:
Webpage Title:
Apple
Webpage Contents:
Apple
Apple
Store
Mac
iPad
iPhone
Watch
Vision
AirPods
TV & Home
Entertainment
Accessories
Support
0
+
Last chance to get AirPods or an eligible accessory of your choice when you buy Mac or iPad with education savings. Ends 9.30.
1
Shop
iPhone 17 Pro
All out Pro.
Learn more
Buy
iPhone Air
The thinnest iPhone ever. With the power of pro inside.
Learn more
Buy
iPhone 17
Magichromatic.
Learn more
Buy
AirPods Pro 3
The world’s best in-ear Active Noise Cancellation.
Learn more
Buy
Apple Watch Series 11
The ultimate way to watch your health.
Learn more
Buy
Apple Watch SE 3
Walk it. Talk it. Track it. Love it.
Learn more
Buy
Apple Watch Ultra 3
Pe

In [44]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [45]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [ ]:
get_brochure_user_prompt("Apple", "https://www.apple.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.apple.com/about/'}, {'type': 'careers page', 'url': 'https://www.apple.com/careers/us/'}, {'type': 'company page', 'url': 'https://www.apple.com/newsroom/'}, {'type': 'leadership page', 'url': 'https://www.apple.com/leadership/'}, {'type': 'environment page', 'url': 'https://www.apple.com/environment/'}, {'type': 'diversity page', 'url': 'https://www.apple.com/diversity/'}, {'type': 'education initiative page', 'url': 'https://www.apple.com/education-initiative/'}, {'type': 'contact page', 'url': 'https://www.apple.com/contact/'}]}


'You are looking at a company called: Apple\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nApple\nWebpage Contents:\nApple\nApple\nStore\nMac\niPad\niPhone\nWatch\nVision\nAirPods\nTV & Home\nEntertainment\nAccessories\nSupport\n0\n+\nLast chance to get AirPods or an eligible accessory of your choice when you buy Mac or iPad with education savings. Ends\xa09.30.\n1\nShop\niPhone 17 Pro\nAll out Pro.\nLearn more\nBuy\niPhone Air\nThe thinnest iPhone ever. With the power of pro inside.\nLearn more\nBuy\niPhone 17\nMagichromatic.\nLearn more\nBuy\nAirPods Pro 3\nThe world’s best in-ear Active Noise Cancellation.\nLearn more\nBuy\nApple Watch Series 11\nThe ultimate way to watch your health.\nLearn more\nBuy\nApple Watch SE 3\nWalk it. Talk it. Track it. Love it.\nLearn more\nBuy\nApple Watch Ultra 3\nPersonal beast.\nLearn more\nBuy\nAny condition carrier\xa0dea

In [47]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [48]:
create_brochure("Apple", "https://www.apple.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.apple.com/about/'}, {'type': 'careers page', 'url': 'https://www.apple.com/careers/us/'}, {'type': 'leadership page', 'url': 'https://www.apple.com/leadership/'}, {'type': 'newsroom page', 'url': 'https://www.apple.com/newsroom/'}, {'type': 'diversity page', 'url': 'https://www.apple.com/diversity/'}, {'type': 'environment page', 'url': 'https://www.apple.com/environment/'}, {'type': 'investor page', 'url': 'https://investor.apple.com/'}]}


```markdown
# Apple Inc. Brochure

## Company Overview
Apple Inc. is a leading technology company renowned for its innovative products that seamlessly blend hardware and software. With a commitment to enhancing the user experience, Apple offers a diverse range of products, including:

- **iPhone**: The latest models, such as the iPhone 17 Pro and iPhone Air, showcase advanced technology and design.
- **iPad**: Offers powerful performance in a portable form.
- **Mac**: A line of personal computers known for their sleek design and reliability.
- **Apple Watch**: A smartwatch that prioritizes health and connectivity.
- **AirPods**: Cutting-edge audio technology with Active Noise Cancellation.
- **Apple TV+**: An entertainment platform featuring original programming.
  
## Vision & Innovation
Apple is driven by a vision of creating innovative products that enrich users' lives. The company prioritizes design, functionality, and sustainability, ensuring that technology is accessible and enjoyable for everyone.

## Company Culture
At Apple, we foster a culture of creativity, collaboration, and excellence. Our employees are encouraged to think outside the box and push boundaries in technology. Diversity and inclusion are key components of our work environment, reflecting our belief that every individual brings unique perspectives and ideas to the table. We support employee growth and innovation by providing opportunities for professional development and encouraging personal projects.

## Customer Commitment
Apple stands firm in its commitment to customer satisfaction. We offer tailored promotions and trade-in programs to make our products even more accessible. Our customer support is designed to assist users anytime, anywhere, ensuring a seamless experience with our products.

### Current Offers:
- **Educational Savings**: Get AirPods or an eligible accessory with the purchase of a Mac or iPad.
- **Trade-In Program**: Receive credits ranging from $200 to $700 when you trade in eligible devices.

## Careers at Apple
Join a passionate team dedicated to innovation! Apple provides vibrant opportunities for individuals at all stages of their careers. We seek applicants who are enthusiastic, forward-thinking, and eager to tackle technology challenges. Whether you're interested in engineering, design, marketing, or customer support, Apple is the place to make your mark.

### Benefits of Working at Apple:
- Competitive salary and benefits
- Opportunities for growth and advancement
- A dynamic, inclusive work environment
- Access to cutting-edge technology and innovative projects

## Get in Touch
Explore our products, career opportunities, and more on our website. Join us in our mission to empower users through technology.

---

*For more information, visit [Apple's Website](https://www.apple.com)*
```


In [49]:
#stream back from OpenAI with the familiar typewriter animation

def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [50]:
stream_brochure("Apple", "https://www.apple.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.apple.com/about/'}, {'type': 'careers page', 'url': 'https://www.apple.com/careers/us/'}, {'type': 'company news', 'url': 'https://www.apple.com/newsroom/'}, {'type': 'leadership page', 'url': 'https://www.apple.com/leadership/'}, {'type': 'investor relations', 'url': 'https://investor.apple.com/'}]}


# Apple Company Brochure

Welcome to Apple, a leading technology company renowned for its innovative products and services. Below are some highlights about our offerings, company culture, and career opportunities.

## Our Products

Apple offers a wide range of products designed to enhance your everyday life and connect you to the world. Our product lineup includes:

- **iPhone**: Discover the latest iPhone models, including the **iPhone 17 Pro**, the **iPhone Air**, and the vibrant **iPhone 17**.
- **iPad**: Power your creativity and productivity with the latest iPad models.
- **Mac**: Advanced computers designed for professionals and casual users alike.
- **Apple Watch**: The ultimate smart watch for tracking health, workouts, and more. Models include the **Apple Watch Series 11**, **Apple Watch SE 3**, and the rugged **Apple Watch Ultra 3**.
- **AirPods**: Experience superior sound quality with our acclaimed **AirPods Pro 3**.
- **Apple TV+**: Enjoy a wide range of entertainment options with original shows and movies.

We are committed to making technology accessible and enjoyable for everyone. 

## Company Culture

At Apple, our culture is driven by innovation, collaboration, and inclusivity. We believe in fostering a dynamic environment where every employee can contribute their talents and ideas. Our core values include:

- **Creativity**: We encourage creative thinking to solve challenges and innovate continuously.
- **Integrity**: We hold ourselves to the highest ethical standards in everything we do.
- **Diversity and Inclusion**: We celebrate diverse backgrounds, perspectives, and experiences, which enhance our work culture and performance.

## Customers

Apple serves millions of customers worldwide, including individuals, businesses, educational institutions, and creative professionals. We prioritize customer satisfaction and strive to provide an exceptional experience with our products and services. 

## Careers at Apple

Apple is a great place to build a career! We are always looking for passionate and talented individuals to join our team. Career opportunities abound across various fields, including:

- Technology and Software Development
- Marketing and Sales
- Customer Support
- Design and User Experience
- Operations and Supply Chain Management

Join Apple and help us shape the future of technology! 

---

For more information, visit our website or contact our support team. Thank you for considering Apple as your next home for innovation, creativity, and growth!